In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\AI & ML\\Gen AI\\Gen-AI-Projects\\End-to-end-Medical-Chatbot-Generative-AI-main\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\AI & ML\\Gen AI\\Gen-AI-Projects\\End-to-end-Medical-Chatbot-Generative-AI-main'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [13]:
# text_chunks

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [26]:
embeddings = download_hugging_face_embeddings()

In [29]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
# query_result

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [39]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "d:\AI & ML\Gen AI\Gen-AI-Projects\End-to-end-Medical-Chatbot-Generative-AI-main\venv\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-pfqor2w.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [40]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [41]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "d:\AI & ML\Gen AI\Gen-AI-Projects\End-to-end-Medical-Chatbot-Generative-AI-main\venv\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


In [42]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "d:\AI & ML\Gen AI\Gen-AI-Projects\End-to-end-Medical-Chatbot-Generative-AI-main\venv\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


In [43]:
docsearch

In [44]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [45]:
retrieved_docs = retriever.invoke("What is Acne?")

In [46]:
retrieved_docs

[Document(id='1e67efe2-69c6-4149-acf0-bfe84703a5b8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='4b41f2f6-ee33-4b7d-b50a-3fb1feccdbf8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [47]:
!pip install langchain-groq

  Using cached langchain_groq-0.2.4-py3-none-any.whl (14 kB)
  Using cached groq-0.18.0-py3-none-any.whl (121 kB)


You should consider upgrading via the 'D:\AI & ML\Gen AI\Gen-AI-Projects\End-to-end-Medical-Chatbot-Generative-AI-main\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [48]:
# from langchain_openai import OpenAI
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0.4, max_tokens=500)

In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [50]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [51]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, resulting in increased growth in bone and soft tissue. It is a rare condition, affecting approximately 50 out of every one million people. The symptoms, such as unusual height and enlarged facial features, develop gradually and often lead to delayed diagnosis.


In [52]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

A complete blood count (CBC) is a series of tests that evaluates the cells suspended in the liquid part of blood, providing valuable information about the blood and other body systems. Abnormal results can indicate conditions such as anemias, leukemias, and infections. It is a useful screening and diagnostic test.
